# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

## 1. Retrieval of the images from GEE

To retrieve from the GEE server the available satellite images cropped around the user-defined region of coastline for the particular time period of interest, the following variables are required:

* Coordinate list: a list of the coordinates of the region of interest (longitude/latitude pairs in WGS84) – see section 'create coordinate list in User handbook to extract ROI from study site
* all_dates: dates over which the images will be retrieved (e.g., dates = ['2017-12-01', '2018-01-01'])
* all_sats: satellite missions to consider (e.g., sat_list = ['L7', 'L8', 'S2'] for Landsat 7, 8 and Sentinel-2 collections)
* sitename: name of the site (this is the name of the subfolder where the images and other accompanying files will be stored)
* filepath: filepath to the directory where the data will be stored

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs) - GEE limits download size

The function `SDS_download.check_images_available(inputs)` will print the number of images used in the median composite. 

In [2]:
coordinate_list =([[-3.11351667599996,5.11984366800004],[-3.11351667599996,5.08507918500004],[-3.23077600099998,5.11984366800004],[-3.23077600099998,5.08507918500004]]),\
([[-3.22619513799998,5.13841693200004],[-3.22619513799998,5.09873313500003],[-3.32753572799993,5.13841693200004],[-3.32753572799993,5.09873313500003]]),\
([[-3.32217515699995,5.15745766700007],[-3.32217515699995,5.11747853300005],[-3.42382558999998,5.15745766700007],[-3.42382558999998,5.11747853300005]]),\
([[-3.42002223499998,5.16876958000006],[-3.42002223499998,5.13661909000007],[-3.52051057599994,5.16876958000006],[-3.52051057599994,5.13661909000007]]),\
([[-3.51779546999995,5.17693327500007],[-3.51779546999995,5.14869023200004],[-3.61806020799997,5.17693327500007],[-3.61806020799997,5.14869023200004]]),\
([[-3.61474489499994,5.19569786200003],[-3.61474489499994,5.16057459900003],[-3.71529174499994,5.19569786200003],[-3.71529174499994,5.16057459900003]]),\
([[-3.71122970199997,5.21644988900005],[-3.71122970199997,5.17882993800004],[-3.81174854799997,5.21644988900005],[-3.81174854799997,5.17882993800004]]),\
([[-3.80692560799997,5.24396163800003],[-3.80692560799997,5.19981515700005],[-3.90702094599993,5.24396163800003],[-3.90702094599993,5.19981515700005]]),\
([[-3.90045023399995,5.26218390500003],[-3.90045023399995,5.22586304200007],[-4.00348849399995,5.26218390500003],[-4.00348849399995,5.22586304200007]]),\
([[-4.00024886899996,5.25676253800003],[-4.00024886899996,5.22591631000006],[-4.10068082599997,5.25676253800003],[-4.10068082599997,5.22591631000006]]),\
([[-4.09784643099994,5.24475852100005],[-4.09784643099994,5.21041626700003],[-4.19838745599998,5.24475852100005],[-4.19838745599998,5.21041626700003]]),\
([[-4.19553041299997,5.22940342800007],[-4.19553041299997,5.20135063900005],[-4.29578010999995,5.22940342800007],[-4.29578010999995,5.20135063900005]]),\
([[-4.29345730199998,5.21897581900004],[-4.29345730199998,5.19266269000002],[-4.39355190199996,5.21897581900004],[-4.39355190199996,5.19266269000002]]),\
([[-4.39158025499995,5.20903141600007],[-4.39158025499995,5.18177177500007],[-4.49176315699998,5.20903141600007],[-4.49176315699998,5.18177177500007]]),\
([[-4.48920003899997,5.19897962000005],[-4.48920003899997,5.17004769600004],[-4.58951604299995,5.19897962000005],[-4.58951604299995,5.17004769600004]]),\
([[-4.58660113999997,5.18845470000002],[-4.58660113999997,5.15436356700008],[-4.68713883499993,5.18845470000002],[-4.68713883499993,5.15436356700008]]),\
([[-4.68407292399996,5.17250113500006],[-4.68407292399996,5.13928243500004],[-4.78459358699996,5.17250113500006],[-4.78459358699996,5.13928243500004]]),\
([[-4.78243233799998,5.15840484200004],[-4.78243233799998,5.13281406600004],[-4.88245330399997,5.15840484200004],[-4.88245330399997,5.13281406600004]]),\
([[-4.88033614099993,5.15098379200003],[-4.88033614099993,5.12442724600004],[-4.98045433899995,5.15098379200003],[-4.98045433899995,5.12442724600004]]),\
([[-4.97844414299993,5.14334099200005],[-4.97844414299993,5.11944628700007],[-5.07827116099998,5.14334099200005],[-5.07827116099998,5.11944628700007]]),\
([[-5.07668631799993,5.13775283800004],[-5.07668631799993,5.11585655300007],[-5.17624734199995,5.13775283800004],[-5.17624734199995,5.11585655300007]]),\
([[-5.17474273899995,5.13309546600004],[-5.17474273899995,5.10852958500004],[-5.27465002199995,5.13309546600004],[-5.27465002199995,5.10852958500004]]),\
([[-5.27262781199994,5.12618324000005],[-5.27262781199994,5.09842873000002],[-5.37285316199996,5.12618324000005],[-5.37285316199996,5.09842873000002]]),\
([[-5.36983905699998,5.11593978200005],[-5.36983905699998,5.08454206100004],[-5.47029711099998,5.11593978200005],[-5.47029711099998,5.08454206100004]]),\
([[-5.46664811699998,5.10349694000007],[-5.46664811699998,5.06533871800002],[-5.56887773199998,5.10349694000007],[-5.56887773199998,5.06533871800002]]),\
([[-5.56499484299997,5.08831440300003],[-5.56499484299997,5.05556366000008],[-5.66550289299994,5.08831440300003],[-5.66550289299994,5.05556366000008]]),\
([[-5.66250946199995,5.07261516000005],[-5.66250946199995,5.04323706800005],[-5.76285598899995,5.07261516000005],[-5.76285598899995,5.04323706800005]]),\
([[-5.75939837499993,5.06108458700004],[-5.75939837499993,5.02453211900007],[-5.85993796599996,5.06108458700004],[-5.85993796599996,5.02453211900007]]),\
([[-5.85488856699993,5.04210327400006],[-5.85488856699993,4.99271604300003],[-5.95424987299998,5.04210327400006],[-5.95424987299998,4.99271604300003]]),\
([[-5.94661727999994,5.00841222600008],[-5.94661727999994,4.95125590800006],[-6.04416397099993,5.00841222600008],[-6.04416397099993,4.95125590800006]]),\
([[-6.03954168799993,4.97860267300007],[-6.03954168799993,4.91252197700004],[-6.13784637099997,4.97860267300007],[-6.13784637099997,4.91252197700004]]),\
([[-6.12536760899997,4.92668797500005],[-6.12536760899997,4.86928958900006],[-6.22284246499993,4.92668797500005],[-6.22284246499993,4.86928958900006]]),\
([[-6.21553754699994,4.88578247300006],[-6.21553754699994,4.83487806900007],[-6.31461575199995,4.88578247300006],[-6.31461575199995,4.83487806900007]]),\
([[-6.30877413899998,4.85293847000003],[-6.30877413899998,4.80708823500004],[-6.40867108099997,4.85293847000003],[-6.40867108099997,4.80708823500004]]),\
([[-6.40209666599998,4.82506188300005],[-6.40209666599998,4.76750904100004],[-6.49952517199995,4.82506188300005],[-6.49952517199995,4.76750904100004]]),\
([[-6.49626732799993,4.79105776300003],[-6.49626732799993,4.75298286600002],[-6.59677330099993,4.79105776300003],[-6.59677330099993,4.75298286600002]]),\
([[-6.58723203599993,4.76275403200003],[-6.58723203599993,4.69894524000006],[-6.68242535099995,4.76275403200003],[-6.68242535099995,4.69894524000006]]),\
([[-6.67696616199993,4.71723096400007],[-6.67696616199993,4.67496989400007],[-6.77723765199994,4.71723096400007],[-6.77723765199994,4.67496989400007]]),\
([[-6.77435696699996,4.69565905900004],[-6.77435696699996,4.66388500700003],[-6.87483093499998,4.69565905900004],[-6.87483093499998,4.66388500700003]]),\
([[-6.86992703299995,4.68656139900003],[-6.86992703299995,4.61449665900005],[-6.96584694299997,4.68656139900003],[-6.96584694299997,4.61449665900005]]),\
([[-6.95140706499996,4.62812966800004],[-6.95140706499996,4.55153179600006],[-7.03933551299997,4.62812966800004],[-7.03933551299997,4.55153179600006]]),\
([[-7.02642498099993,4.56225075100002],[-7.02642498099993,4.51306903400007],[-7.12736295699995,4.56225075100002],[-7.12736295699995,4.51306903400007]]),\
([[-7.12367726299993,4.54093733200006],[-7.12367726299993,4.49708079200008],[-7.22380262399997,4.54093733200006],[-7.22380262399997,4.49708079200008]]),\
([[-7.20963337699993,4.50398671700003],[-7.20963337699993,4.43531715300003],[-7.30587292499996,4.50398671700003],[-7.30587292499996,4.43531715300003]]),\
([[-7.29836432899998,4.45788680600003],[-7.29836432899998,4.38810819900004],[-7.39068593199994,4.45788680600003],[-7.39068593199994,4.38810819900004]]),\
([[-7.38320636599997,4.40463602100004],[-7.38320636599997,4.35128441300003],[-7.48175704499994,4.40463602100004],[-7.48175704499994,4.35128441300003]]),\
([[-7.47955380799993,4.37076705900006],[-7.47955380799993,4.35278396700005],[-7.54039247699995,4.37076705900006],[-7.54039247699995,4.35278396700005]])


counter = 0

for polygon in coordinate_list:

# region of interest (longitude, latitude)
    polygon = coordinate_list[counter]

#IMPORTANT - The code will retrieve the satellite data from the the defined time period and satellite
#            dependant on the number in the list. I.e. ['2000-01-01', '2000-12-31'] or all_dates[0]
#            corresponds to ['L7'] or all_sats[0]. 

    all_dates = ([['2000-01-01', '2000-12-31'],['2001-01-01', '2001-12-31'],['2002-01-01', '2002-12-31'],['2003-01-01', '2003-12-31'],\
                 ['2004-01-01', '2004-12-31'],['2005-01-01', '2005-12-31'],['2006-01-01', '2006-12-31'],['2007-01-01', '2007-12-31'],\
                 ['2008-01-01', '2008-12-31'],['2009-01-01', '2009-12-31'],['2010-01-01', '2010-12-31'],['2011-01-01', '2011-12-31'],\
                 ['2012-01-01', '2012-12-31'],\
                 ['2013-01-01', '2013-12-31'],['2014-01-01', '2014-12-31'],['2015-01-01', '2015-12-31'],\
                 ['2016-01-01', '2016-12-31'],['2017-01-01', '2017-12-31'],['2018-01-01', '2018-12-31'],['2019-01-01', '2019-12-31'],\
                 ['2020-01-01', '2020-12-31']])

    all_sats = ([['L7'],['L7'],['L7'],['L7'],\
                 ['L7'],['L7'],['L7'],['L7'],\
                 ['L7'],['L7'],['L7'],['L7'],\
                 ['L7'],\
                 ['L8'],['L8'],['L8'],\
                 ['S2'],['S2'],['S2'],['S2'],\
                 ['S2']])
    
    rolling = 0

    for dates in all_dates:
        # date range
        dates = all_dates[rolling]
        # satellite missions
        sat_list = all_sats[rolling]
        # name of the site
        foldernumber = counter
        sitename = 'Cote_divoire_' + str(foldernumber) 
        # directory where the data will be stored
        filepath = os.path.join(os.getcwd(), 'data')
        # put all the inputs into a dictionnary
        inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath': filepath, 'rolling': str(rolling)}

        # before downloading the images, check how many images are available for your inputs
        SDS_download.check_images_available(inputs);
        
        SDS_download.Landsat_Coregistration(inputs);
        
        settings = { 
            # general parameters:
            'cloud_thresh': 0.4,        # threshold on maximum cloud cover
            'output_epsg': 2043,        # epsg code of spatial reference system desired for the output   
            # add the inputs defined previously
            'inputs': inputs,
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 1000,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
            'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
        }

        inputs['include_T2'] = False
        metadata = SDS_download.retrieve_images(inputs,settings)

        metadata = SDS_download.get_metadata(inputs)
          
        rolling = rolling + 1
##Batch shoreline detection
    %matplotlib qt
    output = SDS_shoreline.extract_shorelines(metadata, settings)

    counter = counter + 1

Images available between 2000-01-01 and 2000-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 5 images
  Total: 5 images
Displacement Calculated
Images available between 2000-01-01 and 2000-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 5 images
  Total: 5 images

Median Processed
Displacement Calculated
Registered
Downloaded

Images available between 2001-01-01 and 2001-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 9 images
  Total: 9 images
Displacement Calculated
Images available between 2001-01-01 and 2001-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 9 images
  Total: 9 images

Median Processed
Displacement Calculated
Registered
Downloaded

Images available between 2002-01-01 and 2002-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 12 images
  Total: 12 images
Displacement Calculated
Images available between 2002-01-01 and 2002-12-31:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L7: 12 images
  Total: 12 images

Median Processed
Displace

KeyError: 'cloud_thresh'